<a href="https://colab.research.google.com/github/hwang-sudang/kkoApplemusic/blob/simple_tags/0719_tag_sparse_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tag Sparse Matrix
1. Top 300 sparse mat 
2. Khaiii Plyst name sparse matrix
  - 태그 정확도는 0.161075


## Khaiii 초기 설정
!git clone https://github.com/kakao/khaiii.git

### khaii install

In [ ]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

### import libraries

In [ ]:
import json
import pandas as pd
import numpy as np
import re
from collections import Counter
from typing import *

# import from pandas.plotting import register_matplotlib_converters
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pyplot as plt
# from khaiii import KhaiiiApi  #khaiii 레포는 https://github.com/kakao/khaiii 이쪽

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# 한글 폰트 설치해 경로설정해주기 
#!apt -qq -y install fonts-nanum > /dev/null # 폰트 나눔을 구글코랩에 설치하고 경로를 파악함
import matplotlib.font_manager as fm 
fontpath='/user/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font=fm.FontProperties(fname=fontpath,size=9)

##구글 코랩 데이터 갖고오기

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import os
os.chdir("/content/gdrive/My Drive/melon-playlist-continuation-master")
print(os.getcwd())
!ls

/content/gdrive/My Drive/melon-playlist-continuation-master
 0705Tags.ipynb			   README.md
'0716 Khaiii main keyword.ipynb'   requirements.txt
'0719 tag_sparse matrix.ipynb'	   results.json
'0720 tag_sparse_matrix.ipynb'	   song_meta.json
 arena_util.py			  'sparse matrix.ipynb'
 build				   split_data.ipynb
 evaluate.py			   split_data.py
 genre_gn_all.json		   test01.rtf
 genre_most_popular.py		   test.json
 khaiii				   testPickleFile.pickle
 LICENSE			   toptag.csv
 most_popular.py		   train.json
 NOTICE.md			   uniq_tags.csv
 PlaylistEmbedding.ipynb	   uniq_tags.gsheet
 plylst_tags.csv		   uniq_tags.xml
 plylst_top_tag.csv		   val.json
 __pycache__


# 1. Top 300 tag 이용하기

In [ ]:
def readJson():
    train = pd.read_json('train.json', typ = 'frame')
    song_meta = pd.read_json('song_meta.json', typ = 'frame')
    return train, song_meta

train, song_meta = readJson()  # train, song_meta load

In [ ]:
plylst_tag_map = train[['id', 'tags']]
plylst_tag_map.head()

,id,tags
0,61281,[락]
1,10532,"[추억, 회상]"
2,76951,"[까페, 잔잔한]"
3,147456,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,..."
4,27616,[댄스]


In [ ]:
# 리스트 안에 들어있는 태그들을 모두 꺼내기
plylst_tag_map_unnest = np.dstack(
    (
        np.repeat(plylst_tag_map.id.values, list(map(len, plylst_tag_map.tags))), 
        np.concatenate(plylst_tag_map.tags.values) # 기존 어레이를 따라 일련의 어레이 결합
    )
)
plylst_tag_map_unnest

array([[['61281', '락'],
        ['10532', '추억'],
        ['10532', '회상'],
        ...,
        ['100389', '팝송추천'],
        ['100389', '팝송'],
        ['100389', '팝송모음']]], dtype='<U21')

In [ ]:
def plyst_tag_map():
  plylst_tag_map = train[['id', 'tags']]
  plylst_tag_map_unnest=np.dstack(
      (
          np.repeat(plylst_tag_map.id.values, list(map(len, plylst_tag_map.tags))), 
          np.concatenate(plylst_tag_map.tags.values) # 기존 어레이를 따라 일련의 어레이 결합
      )
  )
  plylst_tag_map = pd.DataFrame(data = plylst_tag_map_unnest[0], columns = plylst_tag_map.columns)
  del plylst_tag_map_unnest
  return plylst_tag_map

plylst_tags=plyst_tag_map()
plylst_tags




,id,tags
0,61281,락
1,10532,추억
2,10532,회상
3,76951,까페
4,76951,잔잔한
...,...,...
476326,131982,퇴근길
476327,100389,노래추천
476328,100389,팝송추천
476329,100389,팝송


In [ ]:
# plylst_tags.to_csv("plylst_tags.csv")

In [ ]:
def toptag_n(n):
  plylst_tag_map2=plyst_tag_map()
  tag_list_cnt = plylst_tag_map2.groupby('tags').id.nunique().reset_index(name = 'tag_cnt')
  toptag_n = tag_list_cnt.nlargest(n, 'tag_cnt')
  return toptag_n


In [ ]:
toptag=toptag_n(300).reset_index().drop(["index"],axis='columns')
print(toptag['tags'])
toptag

0          기분전환
1            감성
2            휴식
3           발라드
4           잔잔한
         ...   
295         커피숍
296    EDMFloor
297          동요
298         센치한
299      싱어송라이터
Name: tags, Length: 300, dtype: object


,tags,tag_cnt
0,기분전환,16465
1,감성,11417
2,휴식,11215
3,발라드,10796
4,잔잔한,10218
...,...,...
295,커피숍,163
296,EDMFloor,161
297,동요,161
298,센치한,161


In [ ]:
#toptag.to_csv("toptag.csv")

## 가장 많이 쓰이는 태그 300개 추출

In [ ]:
plylst_in_tag=[]

# 이상하게 함수 안에 들어가면 결과가 달라짐.
for tag in toptag["tags"]:
    a=[]
    for i,j in enumerate(plylst_tags["tags"]):
      if j == tag :
        a.append(plylst_tags["id"][i])
    plylst_in_tag.append(a)

In [ ]:
plylst_in_tag=pd.DataFrame(np.array(plylst_in_tag),columns=["plylst"])
plylst_in_tag 

,plylst
0,"[69252, 151693, 93275, 53737, 105328, 16037, 3..."
1,"[1516, 122843, 131098, 116573, 16037, 75625, 1..."
2,"[1239, 131098, 92257, 61024, 13100, 118405, 13..."
3,"[74655, 16037, 132680, 34198, 140334, 136486, ..."
4,"[76951, 36557, 12460, 122843, 6143, 107911, 69..."
...,...
295,"[94008, 20820, 12354, 33645, 34610, 33091, 726..."
296,"[26774, 80953, 10685, 56730, 103160, 7383, 472..."
297,"[9180, 125723, 115862, 137064, 2978, 133328, 4..."
298,"[11976, 13553, 149618, 29779, 67289, 47020, 35..."


In [ ]:
plylst_top_tag=toptag.join(plylst_in_tag, on=toptag.index, how='inner').drop(["key_0","tag_cnt"],axis="columns")
plylst_top_tag # 각 태그와 플레이리스트를 묶었던 것을 보여줌


,tags,plylst
0,기분전환,"[69252, 151693, 93275, 53737, 105328, 16037, 3..."
1,감성,"[1516, 122843, 131098, 116573, 16037, 75625, 1..."
2,휴식,"[1239, 131098, 92257, 61024, 13100, 118405, 13..."
3,발라드,"[74655, 16037, 132680, 34198, 140334, 136486, ..."
4,잔잔한,"[76951, 36557, 12460, 122843, 6143, 107911, 69..."
...,...,...
295,커피숍,"[94008, 20820, 12354, 33645, 34610, 33091, 726..."
296,EDMFloor,"[26774, 80953, 10685, 56730, 103160, 7383, 472..."
297,동요,"[9180, 125723, 115862, 137064, 2978, 133328, 4..."
298,센치한,"[11976, 13553, 149618, 29779, 67289, 47020, 35..."


### pickles

In [ ]:
#import pickle
import gzip

with gzip.open('testPickleFile.pickle', 'wb') as f:
    pickle.dump(plylst_top_tag, f)

In [ ]:
plylst_top_tag1=np.array(plylst_top_tag["plylst"])
plylst_top_tag1[:10]

array([list(['69252', '151693', '93275', '53737', '105328', '16037', '33019', '124665', '118405', '149384', '140918', '60834', '33025', '150721', '119639', '47522', '15630', '27538', '140912', '73145', '46739', '41361', '101383', '13041', '47202', '82711', '115945', '130509', '66111', '50947', '53394', '55459', '109637', '80519', '303', '19949', '60173', '23153', '103896', '61085', '69400', '49122', '13168', '87082', '35721', '75797', '132283', '126211', '126168', '85656', '67653', '8910', '114870', '50790', '59922', '42641', '15557', '134690', '139372', '50775', '28522', '121092', '54232', '130829', '56235', '57233', '127491', '98405', '151186', '22311', '74259', '20945', '125661', '20086', '52748', '133292', '68311', '57785', '51317', '24466', '18609', '9323', '53485', '65121', '14560', '69978', '65515', '61354', '143427', '17625', '74047', '118137', '45002', '33683', '77', '40380', '82444', '70383', '144386', '116978', '23720', '51226', '36793', '13516', '20234', '114568', '17608', 

# 2. Tags by plylst name
## using Khaiii tokenizer
- 포럼에서 올라온 플레이리스트 이름으로 태그 맞추기 활용


In [ ]:
def re_sub(series: pd.Series) -> pd.Series:
    series = series.str.replace(pat=r'[ㄱ-ㅎ]', repl=r'', regex=True)  # ㅋ 제거용
    series = series.str.replace(pat=r'[^\w\s]', repl=r'', regex=True)  # 특수문자 제거
    series = series.str.replace(pat=r'[ ]{2,}', repl=r' ', regex=True)  # 공백 제거
    series = series.str.replace(pat=r'[\u3000]+', repl=r'', regex=True)  # u3000 제거
    return series

def flatten(list_of_list : List) -> List:
    flatten = [j for i in list_of_list for j in i]
    return flatten

def get_token(title: str, tokenizer)-> List[Tuple]:
    
    if len(title)== 0 or title== ' ':  # 제목이 공백인 경우 tokenizer에러 발생
        return []
    
    result = tokenizer.analyze(title)
    result = [(morph.lex, morph.tag) for split in result for morph in split.morphs]  # (형태소, 품사) 튜플의 리스트
    return result

def get_all_tags(df) -> List:
    tag_list = df['tags'].values.tolist()
    tag_list = flatten(tag_list)
    return tag_list

In [ ]:
tokenizer = KhaiiiApi()
all_tag = get_all_tags(train)
token_tag = [get_token(x, tokenizer) for x in all_tag]

In [ ]:
token_itself = list(filter(lambda x: len(x)==1, token_tag))  # 태그 자체가 형태소여서 분리되지 않는 태그만 골라봅니다
token_itself = flatten(token_itself) # 태그 자체가 형태소여서 분리되지 않는 태그
flatten_token = flatten(token_tag) # 전체적인 태그

In [ ]:
pd.DataFrame(token_itself)

,0,1
0,락,MAG
1,추억,NNG
2,회상,NNG
3,까페,NNG
4,연말,NNG
...,...,...
333088,퇴근버스,NNG
333089,Pop,SL
333090,풍경,NNG
333091,퇴근길,NNG


In [ ]:
pd.DataFrame(flatten_token)

,0,1
0,락,MAG
1,추억,NNG
2,회상,NNG
3,까페,NNG
4,잔잔,XR
...,...,...
722855,팝송,NNG
722856,추천,NNG
722857,팝송,NNG
722858,팝송,NNG


### 플레이리스트 제목을 형태소 분석
- NNG, SL, NNP, MAG,SN가 압도적으로 많습니다.
- 각각 일반 명사, 외국어, 고유 명사, 일반 부사, 숫자 입니다.
- 확인해 보면 숫자의 경우 7080, 2000 같은 태그가 꽤 있습니다.
- 제목을 형태소 분석한 결과가 위의 품사인 경우에만 남겨두겠습니다.

In [ ]:
# 1. 
train['plylst_title'] = re_sub(train['plylst_title'])
train.loc[:, 'ply_token'] = train['plylst_title'].map(lambda x: get_token(x, tokenizer))

# 2.
using_pos = ['NNG','SL','NNP','MAG','SN']  # 일반 명사, 외국어, 고유 명사, 일반 부사, 숫자
train['ply_token'] = train['ply_token'].map(lambda x: list(filter(lambda x: x[1] in using_pos, x)))

# 3.
unique_tag = set(token_itself)
unique_word = [x[0] for x in unique_tag]



정답 tags의 형태소만 남기는 작업

In [ ]:
# 우리의 목적은 정답 tags를 맞추는 것이기 때문에 정답 tags에 나온 형태소만 남겨둡니다.
train['ply_token'] = train['ply_token'].map(lambda x: list(filter(lambda x: x[0] in unique_word, x)))
train.head(10)

,tags,id,plylst_title,songs,like_cnt,updt_date,ply_token
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,"[(여행, NNG), (음악, NNG)]"
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,"[(요즘, NNG), (말, NNG)]"
2,"[까페, 잔잔한]",76951,편하게 잔잔하게 들을 수 있는 곡,"[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,"[(곡, NNG)]"
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,"[(크리스마스, NNG), (분위기, NNG), (흠뻑, MAG)]"
4,[댄스],27616,추억의 노래,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,"[(추억, NNG), (노래, NNG)]"
5,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...",69252,2017 Pop Trend,"[418694, 222305, 96545, 135950, 304687, 457451...",435,2017-09-15 15:59:26.000,"[(2017, SN), (Pop, SL)]"
6,"[짝사랑, 취향저격, 슬픔, 고백, 사랑, 이별]",45339,짝사랑고백사랑이별슬픔 감성을 자극하는곡들,"[528130, 505036, 398947, 627363, 37748, 376358...",24,2016-10-24 21:30:17.000,"[(짝사랑, NNG), (감성, NNG), (자극, NNG), (곡, NNG)]"
7,"[잔잔한, 추억, 회상]",36557,멍청이 내맘도 몰라,"[496913, 632529, 501426, 515574, 411161, 10341...",5,2008-09-23 22:32:02.000,"[(맘, NNG)]"
8,"[일렉트로니카, 포크, 메탈, 락, 댄스, 인디]",70741,DANCING IN THE MOONLIGHT 01,"[634861, 270738, 163936, 692209, 449477, 56342...",0,2019-11-30 21:17:59.000,"[(01, SN)]"
9,"[록, Metal, 이일우, M에센셜, 메탈, Rock, 락]",10288,록메탈 Written by 이일우,"[613315, 439294, 230806, 497932, 113479, 93506...",7,2016-06-17 18:23:54.000,"[(록메탈, NNP), (이일우, NNP)]"


In [ ]:
train2= train[["id","songs","ply_token"]]
train2.head(5)

,id,songs,ply_token
0,61281,"[525514, 129701, 383374, 562083, 297861, 13954...","[(여행, NNG), (음악, NNG)]"
1,10532,"[432406, 675945, 497066, 120377, 389529, 24427...","[(요즘, NNG), (말, NNG)]"
2,76951,"[83116, 276692, 166267, 186301, 354465, 256598...","[(곡, NNG)]"
3,147456,"[394031, 195524, 540149, 287984, 440773, 10033...","[(크리스마스, NNG), (분위기, NNG), (흠뻑, MAG)]"
4,27616,"[159327, 553610, 5130, 645103, 294435, 100657,...","[(추억, NNG), (노래, NNG)]"


# Sparse Matrix 만들기
- 가능하면 sparse tensor 도 만들기
- https://blog.naver.com/wideeyed/221182428460

In [ ]:
def make_tag_mat(plylst):
    index_list=[]
    for l in range(300):
        index_list = index_list + [train['id']]
        
    song_mat=pd.DataFrame(data=[np.zeros(plylst_top_tag.shape[0])], index=[index_list],
                          columns=list(range(plylst_top_tag.shape[0])))
    plylst_songs = train["id"]
    songs_removed = list(range(plylst_top_tag.shape[0]))
    
    for i, song in enumerate(plylst_songs):
        song_mat.iloc[i][song] = 1
        songs_removed.remove(song)
    
    rand=random.sample(songs_removed, 300-len(plylst_songs)) 
    
    for i, song in enumerate(rand):
        song_mat.iloc[len(plylst_songs)+i][song] = 1
        
    return sparse.csr_matrix(song_mat)

In [ ]:
# 0 행렬 만들기
index_list=train['id']
song_mat=pd.DataFrame(data=[np.zeros(plylst_top_tag.shape[0])], index=[index_list],
                          columns=list(range(plylst_top_tag.shape[0])))
song_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
61281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
song_mat.shape

(115071, 300)

In [ ]:
song_mat.index=song_mat.index.get_level_values(0)
song_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
61281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
plylst_top_tag.head()

,tags,plylst
0,기분전환,"[69252, 151693, 93275, 53737, 105328, 16037, 3..."
1,감성,"[1516, 122843, 131098, 116573, 16037, 75625, 1..."
2,휴식,"[1239, 131098, 92257, 61024, 13100, 118405, 13..."
3,발라드,"[74655, 16037, 132680, 34198, 140334, 136486, ..."
4,잔잔한,"[76951, 36557, 12460, 122843, 6143, 107911, 69..."


In [ ]:
song_mat.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
61281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
song_mat.index

Int64Index([ 61281,  10532,  76951, 147456,  27616,  69252,  45339,  36557,
             70741,  10288,
            ...
             53627,   2531, 120713, 145872, 135687, 120325, 106976,  11343,
            131982, 100389],
           dtype='int64', name='id', length=115071)

In [ ]:
for i in plylst_top_tag.index: # i 는 tag index
  for j in plylst_top_tag["plylst"][i]:
    k=f"{j}"
k

'90216'

In [ ]:
song_mat.loc[61281][0]=0
song_mat.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
61281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
plylst_top_tag

,tags,plylst
0,기분전환,"[69252, 151693, 93275, 53737, 105328, 16037, 3..."
1,감성,"[1516, 122843, 131098, 116573, 16037, 75625, 1..."
2,휴식,"[1239, 131098, 92257, 61024, 13100, 118405, 13..."
3,발라드,"[74655, 16037, 132680, 34198, 140334, 136486, ..."
4,잔잔한,"[76951, 36557, 12460, 122843, 6143, 107911, 69..."
...,...,...
295,커피숍,"[94008, 20820, 12354, 33645, 34610, 33091, 726..."
296,EDMFloor,"[26774, 80953, 10685, 56730, 103160, 7383, 472..."
297,동요,"[9180, 125723, 115862, 137064, 2978, 133328, 4..."
298,센치한,"[11976, 13553, 149618, 29779, 67289, 47020, 35..."


In [ ]:
for i in plylst_top_tag.index: # i 는 tag index
  for j in plylst_top_tag["plylst"][i]:
    try:
      song_mat.loc[j][i]=1
    except:
      pass

In [ ]:
song_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
61281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print(song_mat.loc[145].sum()) #198 나와야함

0.0


In [ ]:
song_mat.sum().sum()

0.0

In [ ]:
len(plylst_top_tag['plylst'][254])

198

In [ ]:
help(random.sample)

Help on method sample in module random:

sample(population, k) method of random.Random instance
    Chooses k unique random elements from a population sequence or set.
    
    Returns a new list containing elements from the population while
    leaving the original population unchanged.  The resulting list is
    in selection order so that all sub-slices will also be valid random
    samples.  This allows raffle winners (the sample) to be partitioned
    into grand prize and second place winners (the subslices).
    
    Members of the population need not be hashable or unique.  If the
    population contains repeats, then each occurrence is a possible
    selection in the sample.
    
    To choose a sample in a range of integers, use range as an argument.
    This is especially fast and space efficient for sampling from a
    large population:   sample(range(10000000), 60)



In [ ]:
plylst_top_tag

,tags,plylst
0,기분전환,"[69252, 151693, 93275, 53737, 105328, 16037, 3..."
1,감성,"[1516, 122843, 131098, 116573, 16037, 75625, 1..."
2,휴식,"[1239, 131098, 92257, 61024, 13100, 118405, 13..."
3,발라드,"[74655, 16037, 132680, 34198, 140334, 136486, ..."
4,잔잔한,"[76951, 36557, 12460, 122843, 6143, 107911, 69..."
...,...,...
295,커피숍,"[94008, 20820, 12354, 33645, 34610, 33091, 726..."
296,EDMFloor,"[26774, 80953, 10685, 56730, 103160, 7383, 472..."
297,동요,"[9180, 125723, 115862, 137064, 2978, 133328, 4..."
298,센치한,"[11976, 13553, 149618, 29779, 67289, 47020, 35..."


In [ ]:
plylst_songs=pd.Series(train.iloc[0].index)
plylst_songs

train['id']

0          61281
1          10532
2          76951
3         147456
4          27616
           ...  
115066    120325
115067    106976
115068     11343
115069    131982
115070    100389
Name: id, Length: 115071, dtype: int64

In [ ]:
train.iloc[0]

tags                                                          [락]
id                                                          61281
plylst_title                                              여행같은 음악
songs           [525514, 129701, 383374, 562083, 297861, 13954...
like_cnt                                                       71
updt_date                                 2013-12-19 18:36:19.000
Name: 0, dtype: object

In [ ]:
def make_song_mat(plylst):
    index_list=[]
    for l in range(300):
        index_list = index_list + [plylst['id']]
        
    song_mat=pd.DataFrame(data=[np.zeros(song_meta.shape[0])], index=[index_list],
                          columns=list(range(song_meta.shape[0])))
    plylst_songs=plylst['songs']
    songs_removed = list(range(song_meta.shape[0]))
    
    for i, song in enumerate(plylst_songs):
        song_mat.iloc[i][song] = 1
        songs_removed.remove(song)
    
    rand=random.sample(songs_removed, 300-len(plylst_songs)) 
    
    for i, song in enumerate(rand):
        song_mat.iloc[len(plylst_songs)+i][song] = 1
        
    return sparse.csr_matrix(song_mat)

In [ ]:
len(train.iloc[0]['songs'])

19

# 태그 sparse matrix 만드는 방법 연구 필요
song sparse matrix를 만드는 방법이랑 다르다.

In [ ]:
def make_song_mat(plylst):
    index_list=[]
    for l in range(300):
        index_list = index_list + [plylst['id']]
        
    song_mat=pd.DataFrame(data=[np.zeros(song_meta.shape[0])], index=[index_list],
                          columns=list(range(song_meta.shape[0])))
    plylst_songs=plylst['songs']
    songs_removed = list(range(song_meta.shape[0]))
    
    for i, song in enumerate(plylst_songs):
        song_mat.iloc[i][song] = 1
        songs_removed.remove(song)
    
    rand=random.sample(songs_removed, 300-len(plylst_songs)) 
    
    for i, song in enumerate(rand):
        song_mat.iloc[len(plylst_songs)+i][song] = 1
        
    return sparse.csr_matrix(song_mat)

In [ ]:
a=make_song_mat(train.iloc[0])

NameError: ignored